# Link two datasets
## Introduction

This example shows how two datasets with data about persons can be linked. We will try to link the data based on attributes like first name, surname, sex, date of birth, place and address. The data used in this example is part of [Febrl](https://sourceforge.net/projects/febrl/) and is fictitious. 

First, start with importing the ``recordlinkage`` module. The submodule ``recordlinkage.datasets`` contains several datasets that can be used for testing. For this example, we use the Febrl datasets 4A and 4B. These datasets can be loaded with the function ``load_febrl4``.

In [1]:
%precision 5

from __future__ import print_function

import pandas as pd
pd.set_option('precision',5)
pd.options.display.max_rows = 10


In [2]:
import recordlinkage as rl
from recordlinkage.datasets import load_febrl4

The datasets are loaded with the following code. The returned datasets are of type ``pandas.DataFrame``. This makes it easy to manipulate the data if desired. For details about data manipulation with ``pandas``, see their comprehensive documentation http://pandas.pydata.org/. 

In [3]:
dfA, dfB = load_febrl4()

dfA

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
...,...,...,...,...,...,...,...,...,...,...
rec-2153-org,annabel,grierson,97,mclachlan crescent,lantana lodge,broome,2480,nsw,19840224,7676186
rec-1604-org,sienna,musolino,22,smeaton circuit,pangani,mckinnon,2700,nsw,19890525,4971506
rec-1003-org,bradley,matthews,2,jondol place,horseshoe ck,jacobs well,7018,sa,19481122,8927667


## Make record pairs

It is very intuitive to start with comparing each record in DataFrame ``dfA`` with all records of DataFrame ``dfB``. In fact, we want to make record pairs. Each record pair should contain one record of ``dfA`` and one record of ``dfB``. This process of making record pairs is also called 'indexing'. With the ``recordlinkage`` module, indexing is easy. First, load the ``Pairs`` class. This class takes the dataframes as arguments. In case of deduplication of a single dataframe, one dataframe is sufficient as input argument. 

In [4]:
pcl = rl.Pairs(dfA, dfB)
pairs = pcl.full()

With the method ``Pairs.full``, all possible (and unique) record pairs are made. The method returns a ``pandas.MultiIndex``. The number of pairs is equal to the number of records in ``dfA`` times the number of records in ``dfB``.

In [5]:
print (len(dfA), len(dfB), len(pairs))

5000 5000 25000000


Many of these record pairs do not belong to the same person. In case of one-to-one matching, the number of matches should be no more than the number of records in the smallest dataframe. In case of full indexing, ``min(len(dfA), len(N_dfB))`` is much smaller than ``len(pairs)``. The ``recordlinkage`` module has some more advanced indexing methods to reduce the number of record pairs. Obvious non-matches are left out of the index. Note that if a matching record pair is not included in the index, it can not be matched anymore.

One of the most well known indexing methods is named *blocking*. This method includes only record pairs that are identical on one or more stored attributes of the person (or entity in general). The blocking method can be used in the ``recordlinkage`` module. 

In [6]:
pairs = pcl.block('given_name')

print (len(pairs))

77249


The argument 'given_name' is the blocking variable. This variable has to be the name of a column in ``dfA`` and ``dfB``. It is possible to parse a list of columns names to block on multiple variables. Blocking on multiple variables will reduce the number of record pairs even further. 

Another implemented indexing method is *Sorted Neighbourhood Indexing* (``Pairs.sortedneighbourhood``). This method is very useful when there are many misspellings in the string were used for indexing. In fact, sorted neighbourhood indexing is a generalisation of blocking. See the documentation for details about sorted neighbourd indexing.

## Compare records

Each record pair is a candidate match. To classify the candidate record pairs into matches and non-matches, compare the records on all attributes both records have in common. The ``recordlinkage`` module has a class named ``Compare``. This class is used to compare the records. The following code shows how to compare attributes. 

In [7]:
# This cell can take some time to compute. 
compare_cl = rl.Compare(pairs, dfA, dfB)

compare_cl.exact('given_name', 'given_name', name='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, name='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', name='date_of_birth')
compare_cl.exact('suburb', 'suburb', name='suburb')
compare_cl.exact('state', 'state', name='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, name='address_1');

All attribute comparisons are stored in a DataFrame with horizontally the features and vertically the record pairs. The comparison can be found in ``vectors`` attribute of the ``Compare`` class. The first 10 comparison vectors are:

In [8]:
compare_cl.vectors

given_name  surname  date_of_birth  suburb  \
rec_id       rec_id                                                       
rec-1070-org rec-3024-dup-0           1        0              0       0   
             rec-2371-dup-0           1        0              0       0   
             rec-4652-dup-0           1        0              0       0   
             rec-4795-dup-0           1        0              0       0   
             rec-1314-dup-0           1        0              0       0   
...                                 ...      ...            ...     ...   
rec-4528-org rec-4528-dup-0           1        1              1       1   
rec-4887-org rec-4887-dup-0           1        1              1       0   
rec-4350-org rec-4350-dup-0           1        1              1       1   
rec-4569-org rec-4569-dup-0           1        1              1       1   
rec-3125-org rec-3125-dup-0           1        1              1       0   

                             state  address_1  
rec_id       rec_id                            
rec-1070-org rec-3024-dup-0      1          0  
             rec-2371-dup-0      0          0  
             rec-4652-dup-0      0          0  
             rec-4795-dup-0      1          0  
             rec-1314-dup-0      1          0  
...                            ...        ...  
rec-4528-org rec-4528-dup-0      1          1  
rec-4887-org rec-4887-dup-0      1          1  
rec-4350-org rec-4350-dup-0      1          1  
rec-4569-org rec-4569-dup-0      1          0  
rec-3125-org rec-3125-dup-0      1          1  

[77249 rows x 6 columns]

In [9]:
compare_cl.vectors.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,77249.0,77249.00000,77249.00000,77249.00000,77249.00000,77249.00000
mean,1.0,0.04443,0.03793,0.03226,0.24877,0.03670
std,0.0,0.20604,0.19103,0.17669,0.43230,0.18802
min,1.0,0.00000,0.00000,0.00000,0.00000,0.00000
25%,1.0,0.00000,0.00000,0.00000,0.00000,0.00000
50%,1.0,0.00000,0.00000,0.00000,0.00000,0.00000
75%,1.0,0.00000,0.00000,0.00000,0.00000,0.00000
max,1.0,1.00000,1.00000,1.00000,1.00000,1.00000


The last step is to decide which records belong to the same person. In this example, we keep it simple:

In [10]:
# Sum the comparison results.
compare_cl.vectors.sum(axis=1).value_counts().sort_index(ascending=False)

6     1566
5     1332
4      343
3      146
2    16427
1    57435
dtype: int64

In [11]:
compare_cl.vectors[compare_cl.vectors.sum(axis=1) > 3]

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id,rec_id,,,,,,
rec-2371-org,rec-2371-dup-0,1,1,1,1,1,1
rec-3024-org,rec-3024-dup-0,1,1,1,0,1,0
rec-4652-org,rec-4652-dup-0,1,1,1,0,1,1
rec-4795-org,rec-4795-dup-0,1,1,1,1,1,1
rec-1016-org,rec-1016-dup-0,1,1,1,1,0,1
...,...,...,...,...,...,...,...
rec-4528-org,rec-4528-dup-0,1,1,1,1,1,1
rec-4887-org,rec-4887-dup-0,1,1,1,0,1,1
rec-4350-org,rec-4350-dup-0,1,1,1,1,1,1


## Full code

In [12]:
import recordlinkage as rl
from recordlinkage.datasets import load_febrl4

dfA, dfB = load_febrl4()

# Indexation step
pcl = rl.Pairs(dfA, dfB)
pairs = pcl.block('given_name')

# Comparison step
compare_cl = rl.Compare(pairs, dfA, dfB)

compare_cl.exact('given_name', 'given_name', name='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, name='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', name='date_of_birth')
compare_cl.exact('suburb', 'suburb', name='suburb')
compare_cl.exact('state', 'state', name='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, name='address_1')

# Classification step
matches = compare_cl.vectors[compare_cl.vectors.sum(axis=1) > 3]
print(len(matches))


3241
